In [110]:
class Vertex:
    def __init__(self, element):
        self.element = element
        
    def __mul__(self, other):
        return self.element

    def incidentEdges(self, edges):
        incident_edges = []
        for edge in edges:
            if edge.isIncidentOn(self):
                incident_edges.append(edge)
        return incident_edges

    def isAdjacentTo(self, vertex):
        return self.element == vertex.element

class Edge:
    def __init__(self, vertex1, vertex2, element):
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.element = element

    def __mul__(self, other):
        return self.element

    def endVertices(self):
        return (self.vertex1, self.vertex2)

    def opposite(self, vertex):
        if vertex == self.vertex1:
            return self.vertex2
        elif vertex == self.vertex2:
            return self.vertex1
        else:
            raise ValueError("Provided vertex is not incident on this edge.")

    def isAdjacentTo(self, vertex):
        return vertex == self.vertex1 or vertex == self.vertex2

    def isIncidentOn(self, vertex):
        return vertex == self.vertex1 or vertex == self.vertex2

class Graph:
    def __init__(self, filename):
        self.vertices = []
        self.edges = []
        self.insertFromFile(filename)
        
    def getVertex(self, element):
        for vertex in self.vertices:
            if vertex.element == element:
                return vertex
        return None  #

    def vertices(self):
        return self.vertices

    def edges(self):
        return self.edges

    def insertVertex(self, element):
        vertex = Vertex(element)
        self.vertices.append(vertex)

    def insertEdge(self, vertex1, vertex2, element):
        v1 = None
        v2 = None
        for vertex in self.vertices:
            if vertex.isAdjacentTo(vertex1):
                v1 = vertex
            elif vertex.isAdjacentTo(vertex2):
                v2 = vertex

        if v1 is None or v2 is None:
            print("Insertion Failed")
        else:
            edge = Edge(v1, v2, element)
            self.edges.append(edge)
            print("Edge Inserted")

    def eraseVertex(self, vertex):
        incident_edges = vertex.incidentEdges(self.edges)
        for edge in incident_edges:
            self.edges.remove(edge)
        self.vertices.remove(vertex)
        print("Removed")
    def incidentEdges(self, edges, vertex_element):
        incident_edges = []
        for edge in edges:
            if edge.vertex1.element == vertex_element or edge.vertex2.element == vertex_element:
                incident_edges.append(edge)
        return incident_edges
    def eraseEdge(self, edge):
        self.edges.remove(edge)
        print("Removed")
    
    
    def isEdgeExists(self, vertex1_element, vertex2_element):
        vertex1 = self.getVertex(vertex1_element)
        vertex2 = self.getVertex(vertex2_element)

        if vertex1 is None or vertex2 is None:
            return False

        for edge in self.edges:
            if (edge.vertex1 == vertex1 and edge.vertex2 == vertex2) or \
               (edge.vertex1 == vertex2 and edge.vertex2 == vertex1):
                return True

        return False
    
    
    def insertFromFile(self, filename):
        with open(filename, 'r') as file:
            lines = file.readlines()
            vertices_line = lines[0].strip().split(',')
            for vertex_element in vertices_line:
                vertex = Vertex(vertex_element)
                self.vertices.append(vertex)

            for line in lines[1:]:
                line = line.strip().split('\t')
                vertex1 = None
                vertex2 = None
                element = line[1]
                for vertex in self.vertices:
                    if vertex.element == line[0]:
                        vertex1 = vertex
                    elif vertex.element == line[2]:
                        vertex2 = vertex

                if vertex1 is not None and vertex2 is not None:
                    self.insertEdge(vertex1, vertex2, element)
                    
    def hasVertex(self, element):
        """
        Check if a vertex with the given element exists in the graph.

        Args:
            element (str): The element to check.

        Returns:
            bool: True if the vertex exists, False otherwise.
        """
        for vertex in self.vertices:
            if vertex.element == element:
                return True
        return False

    def findPath(self, start_vertex, end_vertex, visited=None, path=None):
        if visited is None:
            visited = set()
        if path is None:
            path = []

        visited.add(start_vertex.element)
        path.append(start_vertex.element)

        if start_vertex == end_vertex:
            return path

        for edge in self.incidentEdges(self.edges, start_vertex.element):
            if edge.vertex1.element not in visited:
                result = self.findPath(edge.vertex1, end_vertex, visited, path)
                if result is not None:
                    return result
            if edge.vertex2.element not in visited:
                result = self.findPath(edge.vertex2, end_vertex, visited, path)
                if result is not None:
                    return result

        path.pop()
        return None

    
def main():
    filename = input("Enter the filename: ")
    g = Graph(filename)
    print("Graph is ready.")

    while True:
        print("Please choose an option:")
        print("1. Find edges incident on a vertex")
        print("2. Find a path in the graph")
        print("3. Insert an edge")
        print("4. Erase a vertex")
        print("5. Quit")
        
        choice = input("Enter your choice: ")
        if choice == '1':
            vertex_element = input("Enter the label of the vertex: ")
            vertex = g.getVertex(vertex_element)
            if vertex:
                incident_edges = vertex.incidentEdges(g.edges)
                print(f"Edges incident on vertex {vertex_element}:")
                for edge in incident_edges:
                    print(f"{edge.vertex1.element} to {edge.vertex2.element} is {edge.element} mi")
            else:
                print(f"Vertex {vertex_element} not found.")
        
        elif choice == '2':
            vertex_element1 = input("Enter the label of the starting vertex: ")
            vertex_element2 = input("Enter the label of the ending vertex: ")
            
            
            vertex1 = g.getVertex(vertex_element1)
            vertex2 = g.getVertex(vertex_element2)
            
            
            
            if vertex1 and vertex2:
                path = g.findPath(vertex1, vertex2)
                
                if path:
#                     print(f"Path from {vertex_element1} to {vertex_element2}:")
                    print(" to ".join([vertex for vertex in path]))
                else:
                    print(f"No path found from {vertex_element1} to {vertex_element2}.")
            else:
                print("Invalid vertices. Please provide valid vertices.")
        
        elif choice == '3':
            vertex_element1 = input("Enter the label of the first vertex: ")
            vertex_element2 = input("Enter the label of the second vertex: ")
            
            
            is_presentv1=g.hasVertex(vertex_element1)
            is_presentv2=g.hasVertex(vertex_element2)
            if is_presentv1 is False or is_presentv2 is False:
                print("Edge insert failed: vertices not exist")
                continue
            if g.isEdgeExists(vertex_element1,vertex_element2)==True:
                print("Edge insert failed: edge  already exists")
                continue
            
            
            
            element = input("Enter the element of the new edge: ")
            vertex1 = g.getVertex(vertex_element1)
            vertex2 = g.getVertex(vertex_element2)
            if vertex1 and vertex2:
                if not g.isEdgeExists(vertex1, vertex2):
                    g.insertEdge(vertex1, vertex2, element)
                    print("Edge Inserted.")
                else:
                    print(f"Edge between {vertex_element1} and {vertex_element2} already exists.")
            else:
                print("Invalid vertices. Please provide valid vertices.")
        
        elif choice == '4':
            vertex_element = input("Enter the label of the vertex to be removed: ")
            vertex = g.getVertex(vertex_element)
            if vertex:
                g.eraseVertex(vertex)
                print(f"Vertex {vertex_element} removed.")
            else:
                print(f"Vertex {vertex_element} not found.")
        
        elif choice == '5':
            print("Quitting...")
            break
        
        else:
            print("Invalid choice. Please choose a valid option.")
main()


In [111]:
main()

Enter the filename: new.txt
Edge Inserted
Edge Inserted
Edge Inserted
Edge Inserted
Edge Inserted
Graph is ready.
Please choose an option:
1. Find edges incident on a vertex
2. Find a path in the graph
3. Insert an edge
4. Erase a vertex
5. Quit
Enter your choice: 1
Enter the label of the vertex: AS
Vertex AS not found.
Please choose an option:
1. Find edges incident on a vertex
2. Find a path in the graph
3. Insert an edge
4. Erase a vertex
5. Quit
Enter your choice: A
Invalid choice. Please choose a valid option.
Please choose an option:
1. Find edges incident on a vertex
2. Find a path in the graph
3. Insert an edge
4. Erase a vertex
5. Quit
Enter your choice: 2
Enter the label of the starting vertex: A
Enter the label of the ending vertex: B
A to B
Please choose an option:
1. Find edges incident on a vertex
2. Find a path in the graph
3. Insert an edge
4. Erase a vertex
5. Quit
Enter your choice: 5
Quitting...
